<a href="https://colab.research.google.com/github/NikolasGialitsis/ProteinsDistanceGeometry/blob/master/ProteinsDistanceGeometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PDB coords extraction

In [32]:
Xtrain_coords = []
model_num = 1
molecules = ['1uwc.pdb','6fat.pdb']
mol_array = []
for mol in molecules:
  print('Get coords for molecule ',mol)
  array = []
  with open(mol) as pdbfile:
      for line in pdbfile:
          #print(line)
          if line[:5] == 'MODEL':
            print('Parsing  Molecule '+str(model_num))
            model_num = model_num + 1
            mol1 = []
          elif line[:4] == 'ATOM':
              #print(line)
              # Split the line
              x_list = line[30:38]
              y_list = line[38:46]
              z_list = line[46:54]

              x_str = ' '.join([str(elem) for elem in x_list]) 
              y_str = ' '.join([str(elem) for elem in y_list]) 
              z_str = ' '.join([str(elem) for elem in z_list]) 
              
              x_str_no_space = x_str.replace(" ", "")
              y_str_no_space = y_str.replace(" ", "")
              z_str_no_space = z_str.replace(" ", "")

              x_float = float(x_str_no_space)
              y_float = float(y_str_no_space)
              z_float = float(z_str_no_space)

              array.append((x_float,y_float,z_float))
      mol_array.append(array)   
      #print('x = '+str(x_float),'y = '+str(y_float), 'z = ' + str(z_float))


Get coords for molecule  1uwc.pdb


FileNotFoundError: ignored

## Calculate c-RMSD

In [164]:
import numpy as np



def euclidean_norm_power2(a,b):
  dim = len(a)
  assert(dim > 0)
  assert(dim == len(b))
  sum_sqrts = 0
  for d in range(dim):
    sum_sqrts += ((a[d] - b[d])*(a[d] - b[d]))
  assert(sum_sqrts >= 0)
  return sum_sqrts

def cRMSD(listX,listY):
  assert(len(listX) == len(listY))
  n = len(listX)
  assert(n > 0)
  sum_atoms = 0
  for ident in range(n):
    x = listX[ident]
    y = listY[ident]
    sum_atoms += euclidean_norm_power2(x,y)
  
  return np.sqrt(sum_atoms/n)

def getCentroid(atomsList):
  sum_atoms = []
  n = len(atomsList)
  dims = len(atomsList[0])
  for dim in range(dims):
    sum_atoms.append(np.array([atom[dim] for atom in atomsList]).sum()/n)
  return sum_atoms

def translateOrigin(atomsList):
  centroid = getCentroid(atomsList)
  for x in atomsList:
    for d in range(len(x)):
      x[d] -= centroid[d]

def GetImproved_cRMSD(X,Y):
  translateOrigin(X)
  translateOrigin(Y)
  XY = np.matmul(X.transpose(),Y)
  SVD = np.linalg.svd(XY)

  U = SVD[0]
  Sigma = SVD[1]
  V = SVD[2]
  print('============ U ============')
  print(U)
  print('============ Σ ============')
  print(Sigma)
  print('============ V ============')
  print(V)
  #assert(U.shape == (3 , 3))
  #assert(Sigma.shape == (3,))
  #assert(V.shape == (3 , 3))
  Q = np.matmul(U,V)

  detQ = np.linalg.det(Q)
  if detQ < 0:
    U[2] = -U[2]
    Q = np.matmul(U,V)
    detQ = np.linalg.det(Q)
    assert(detQ >= 0)
  print('============ Q ============')
  print(Q)

  XQ = np.matmul(X,Q)
  final = cRMSD(XQ,Y)
  assert(final != float("inf"))
  assert(final != float("-inf"))
  assert(final >= 0)
  return final


print(len(mol_array[0]))
print(len(mol_array[1]))
out = GetImproved_cRMSD(np.array(mol_array[0]),np.array(mol_array[1]))
print('\n====================================\nFINAL cRMSD SCORE = ',out)
print('====================================')

CONF.append(confA)
CONF.append(confB)


IndexError: ignored

## (C) construct Cayley-Menger (border) matrix B
Select 3 residues at the calcium binding site namely Val276-Ala277-Asp278 in the 3D
structure of Feruloyl esterase with PDB code 6FAT. Consider only their 3 backbone atoms
(smallest indices) namely atoms N, Ca, C of each, as points with 3D coordinates. Construct
the corresponding Cayley-Menger (border) matrix B of dimension 10x10.

In [87]:
with open('atom_coords.txt',mode='r',encoding='utf-8-sig') as f:
  data = f.readlines()
atoms = []
for line in data:
  x,y,z = line.split()
  atoms.append( (float(x),float(y),float(z)))
print(atoms)

[(19.456, 43.867, 53.82), (19.12, 44.932, 52.867), (19.18, 46.289, 53.576), (20.251, 47.037, 53.302), (20.462, 48.366, 53.881), (19.599, 49.411, 53.172), (18.326, 49.468, 53.563), (17.345, 50.411, 52.999), (16.733, 51.352, 54.051)]


In [43]:
import numpy as np
def BorderMatrix(atoms):
  num_atoms = len(atoms)
  array = -1 * np.ones(shape=(num_atoms+1,num_atoms+1))
  #print(array)
  for row in range(num_atoms+1):
    for col in range(num_atoms+1):
      if row == col:
        array[row,col] = 0
      elif row == 0 or col == 0:
        array[row,col] = 1 
      elif array[col,row] == -1:
        p1 = atoms[row-1]
        p2 = atoms[col-1]
        array[row,col] =  array[col,row] = 0.5* euclidean_norm_power2(p1,p2)
        
  assert(array.all() >= 0)
  return array

B = BorderMatrix(atoms)
print(B)


[[ 0.         1.         1.         1.         1.         1.
   1.         1.         1.         1.       ]
 [ 1.         0.         1.077665   3.000898   5.4746245 10.628379
  15.5881445 16.357075  23.977149  31.7466575]
 [ 1.         1.077665   0.         1.173865   2.9497055  7.310758
  10.1919535 10.845074  16.593745  24.1580125]
 [ 1.         3.000898   1.173865   0.         0.8908105  3.025239
   5.0428305  5.417763  10.345519  15.9237015]
 [ 1.         5.4746245  2.9497055  0.8908105  0.         1.0730015
   3.03894    4.8417535  9.9602605 15.778275 ]
 [ 1.        10.628379   7.310758   3.025239   1.0730015  0.
   1.1697375  2.939012   7.337819  11.4252685]
 [ 1.        15.5881445 10.1919535  5.0428305  3.03894    1.1697375
   0.         0.8883295  3.0552225  6.377039 ]
 [ 1.        16.357075  10.845074   5.417763   4.8417535  2.939012
   0.8883295  0.         1.084853   3.1626245]
 [ 1.        23.977149  16.593745  10.345519   9.9602605  7.337819
   3.0552225  1.084853   0.    

## D) Gram Matrix, SVD
 Check that rank(B)=5. Compute the Gram matrix G, and its SVD so as to get 3D point
coordinates. Check if it is the same structure, by computing its c-RMSD to (c).


In [44]:
rank = np.linalg.matrix_rank(B)
print('Border Matrix Rank = ',rank)
assert(rank==5)

Border Matrix Rank =  5


In [174]:
import numpy as np
def GramMatrix(atoms):
  atoms_append = atoms.copy()
  atoms_append.insert(0,(0,0,0))
  atoms_append = np.array(atoms_append)
  #atoms_append[0,:] = 0
  G = np.dot(np.array(atoms_append),np.array(atoms_append).transpose())
  return G

G =  GramMatrix(atoms)
print(G.round(1))



[[   0.     0.     0.     0.     0.     0.     0.     0.     0.     0. ]
 [   0.  5199.4 5188.3 5287.2 5326.1 5419.7 5410.5 5409.3 5401.2 5487.2]
 [   0.  5188.3 5179.4 5279.  5318.6 5412.9 5405.9 5404.8 5398.6 5484.8]
 [   0.  5287.2 5279.  5380.9 5421.4 5518.  5511.8 5511.  5505.6 5593.8]
 [   0.  5326.1 5318.6 5421.4 5463.7 5561.3 5555.2 5553.  5547.4 5635.3]
 [   0.  5419.7 5412.9 5518.  5561.3 5661.1 5655.8 5653.6 5648.7 5738.4]
 [   0.  5410.5 5405.9 5511.8 5555.2 5655.8 5652.8 5651.5 5648.9 5739.3]
 [   0.  5409.3 5404.8 5511.  5553.  5653.6 5651.5 5651.9 5650.4 5742.1]
 [   0.  5401.2 5398.6 5505.6 5547.4 5648.7 5648.9 5650.4 5651.  5743.6]
 [   0.  5487.2 5484.8 5593.8 5635.3 5738.4 5739.3 5742.1 5743.6 5838.5]]


 Check if it is the same structure, by computing its c-RMSD to (c).


In [195]:
_,Sigma,V = np.linalg.svd(G)
Sigma[3:] = 0
Sigma = np.sqrt(Sigma)
Sigma = np.array(Sigma)
V = np.array(V)
P = Sigma * V
print('Rank of P = ',\
      np.linalg.matrix_rank(P))
print(cRMSD(B,P))

Rank of P =  3
73.34891211208765
